# Data uploading

In [187]:
import json
import pandas as pd
import numpy as np

In [188]:
from tqdm import tqdm
tqdm.pandas()

In [189]:
df = pd.DataFrame()

In [190]:
def get_X_y(data_json_file):
    text_lst, query_lst, ans_lst, y = [], [], [], []
    with open(data_json_file, 'r', encoding="utf-8") as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    text_lst.append(text)
                    query_lst.append(query)
                    ans_lst.append(a['text'])
                    y.append(a['label'])
    return text_lst, query_lst, ans_lst, y

In [191]:
def get_X_test(data_json_file):
    X1 = []
    X2 = []
    X3 = []
    with open(data_json_file, 'r', encoding="utf-8") as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X1.append(text) #текст
                    X2.append(query) #вопрос
                    X3.append(a['text']) #ответ на вопрос
    return X1, X2, X3

In [192]:
text, query, ans, y_train = get_X_y('data/train.jsonl')

train_df = pd.DataFrame({
    'text': text,
    'query': query,
    'ans': ans,
    'label': y_train
})

# в колонке 'text'
# лежит сам текст, вопрос, ответ на этот вопрос
# так как к одному тексту может быть несколько ответов, то похожих строк может быть несколько
train_df.head()

,text,query,ans,label
0,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,В парке.,1
1,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,В лесу.,0
2,"(1) Но люди не могут существовать без природы,...",Где бегала шпана?,Около подъезда.,0
3,"(1) Но люди не могут существовать без природы,...",Почему Люда ударила Артема?,Он к ней приставал.,1
4,"(1) Но люди не могут существовать без природы,...",Почему Люда ударила Артема?,Он ее оскорбил.,0


In [193]:
X1, X2, X3 = get_X_test('data/val_wtihout.jsonl')

test_df = pd.DataFrame({
    'text': X1,
    'query': X2,
    'ans': X3
})

test_df.head()

,text,query,ans
0,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,Так перевозили особо важных заключенных.
1,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что был эмигрантом."
2,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он сам вырыл себе землянку."
3,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он побывал на пересылке Красная Пр..."
4,(1) Самый первый «остров» Архипелага возник в ...,Почему Солженицына перевозили спецконвоем?,"Потому, что он был особо важным заключённым и ..."


# Catboost

In [40]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [70]:
X = train_df[['text', 'query', 'ans']]
y = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y)

text_features = ['text', 'query', 'ans']

model = CatBoostClassifier(
    text_features=text_features,
    use_best_model=False,
    iterations=500,
)

In [ ]:
model.fit(X_train, y_train, eval_set=(X_test, y_test))

In [75]:
test_df['label'] = model.predict(test_df)

In [76]:
test_df = test_df.reset_index()

In [77]:
test_df[['index', 'label']].to_csv('sub.csv', index=False)

# Bert

In [194]:
from IPython.display import clear_output

In [195]:
from torch.utils.data import Dataset, DataLoader

In [196]:
import torch

In [197]:
import torch.nn as nn

In [198]:
from transformers import DistilBertTokenizerFast, BertTokenizer, DistilBertModel, AdamW, get_linear_schedule_with_warmup

In [199]:
from sklearn.metrics import accuracy_score

In [200]:
import matplotlib.pyplot as plt

In [201]:
from copy import deepcopy

In [202]:
from sklearn.model_selection import train_test_split

## Dataset

In [203]:
class ToxicCommentsDataset(Dataset):
    def __init__(
            self,
            data: pd.DataFrame,
            tokenizer: BertTokenizer,
            max_token_len: int=128,
            test=False
        ) -> None:

        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.test = test
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int) -> dict:
        data_row = self.data.iloc[index]
        
        # Извлекаем текстовые поля
        text = data_row['text']
        query = data_row['query']
        ans = data_row['ans']
        
        # Объединяем текст, query и ans в один вход с разделителями [SEP]
        combined_text = f"{text} [SEP] {query} [SEP] {ans}"
        
        # Токенизация объединенного текста
        encoding = self.tokenizer(
            combined_text,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )
        
        # Формируем словарь с одним набором input_ids и attention_mask
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }
        
        # Если это тренировочный или валидационный набор, добавляем метку
        if not self.test:
            item["label"] = torch.tensor(data_row['label'], dtype=torch.long)
        
        return item

In [204]:
BERT_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

In [205]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2008)

In [215]:
# test=False
train_dataset = ToxicCommentsDataset(
  train_df,
  tokenizer,
  max_token_len=128
)

val_dataset = ToxicCommentsDataset(
  val_df,
  tokenizer,
  max_token_len=128,
)

In [216]:
BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Architecture

In [217]:
class Relatable_question(nn.Module):
    def __init__(self, n_classes: int, n_training_steps=None):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)  # загрузка предобученной модели BERT
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)  # добавление линейного слоя
        self.n_training_steps = n_training_steps
        
        # for param in list(self.bert.parameters())[:-4]:
        #     param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        hidden_state = output.last_hidden_state[:, 0, :]
        output = self.classifier(hidden_state)
        return output

In [218]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Relatable_question(n_classes=2).to(DEVICE)


# fitting

In [219]:
def train(model, data_loader, optimizer, loss_fn):
    model = model.to(DEVICE)
    model.train()

    total_loss = 0
    
    y_true = list()
    y_pred = list()

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids']
        at_mask = batch['attention_mask']
        y = batch['label']
        input_ids, at_mask, y = input_ids.to(DEVICE), at_mask.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()

        output = model(input_ids, at_mask)

        loss = loss_fn(output, y)

        loss.backward()

        total_loss += loss.item()
 
        y_true.extend(y.tolist())
        y_pred.extend(output.argmax(dim=1).tolist())

        optimizer.step()

    return total_loss / len(data_loader), accuracy_score(y_true, y_pred)

In [220]:
def evaluate(model, data_loader, loss_fn):
    model = model.to(DEVICE)
    model.eval()

    total_loss = 0
    y_true = list()
    y_pred = list()

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids']
            at_mask = batch['attention_mask']
            y = batch['label']
            input_ids, at_mask, y = input_ids.to(DEVICE), at_mask.to(DEVICE), y.to(DEVICE)
            output = model(input_ids, at_mask)

            loss = loss_fn(output, y)
            total_loss += loss.item()

            y_true.extend(y.tolist())
            y_pred.extend(output.argmax(dim=1).tolist())

    return total_loss / len(data_loader), accuracy_score(y_true, y_pred)


In [221]:
def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    train_accuracy: list[float],
    valid_accuracy: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()

    plt.show()

    plt.figure(figsize=(16, 8))

    plt.title(title + ' Accuracy')

    plt.plot(train_accuracy, label='Train accuracy')
    plt.plot(valid_accuracy, label='Valid accuracy')
    plt.legend()

    plt.show()

In [222]:
def fit(model, train_loader, valid_loader, optimizer, loss_fn, num_epochs, title='Model'):
    train_loss_history, valid_loss_history = [], []
    train_accuracy_history, valid_accuracy_history = [], []

    best_valid_accuracy = 0.0
    best_model = None

    def epoch(count):
        nonlocal best_valid_accuracy, best_model

        train_loss, train_accuracy = train(model, train_loader, optimizer, loss_fn)
        valid_loss, valid_accuracy = evaluate(model, valid_loader, loss_fn)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        train_accuracy_history.append(train_accuracy)
        valid_accuracy_history.append(valid_accuracy)

        clear_output()

        print(f"Epoch: {count}")
        print(f"Accuracy: {valid_accuracy:.4f}")

        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            best_model = deepcopy(model)

    epoch(1)

    for i in range(2, num_epochs + 1):
        epoch(i)

        plot_stats(
            train_loss_history, valid_loss_history,
            train_accuracy_history, valid_accuracy_history,
            title
        )

    return best_model, best_valid_accuracy

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)
fit(model, train_dataloader, val_dataloader, optimizer, criterion, 5)

Epoch: 1
Accuracy: 0.5569


  0%|          | 1/299 [00:00<01:30,  3.28it/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Теперь подстраиваем размер словаря модели под токенизатор
model.bert.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [ ]:
print("Размер словаря токенизатора:", len(tokenizer))
print("Размер словаря модели:", model.bert.config.vocab_size)

Размер словаря токенизатора: 30522
Размер словаря модели: 30522


## Inference

In [ ]:
test_dataset = ToxicCommentsDataset(
  test_df,
  tokenizer,
  max_token_len=128,
  test=True
)

In [ ]:
BATCH_SIZE = 32

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def predict(model, data_loader):
    model = model.to(DEVICE)

    model.eval()
    y_pred = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids']
            at_mask = batch['attention_mask']
            input_ids, at_mask = input_ids.to(DEVICE), at_mask.to(DEVICE)
            output = model(input_ids, at_mask)

            y_pred.extend(output.argmax(dim=1).tolist())

    return y_pred

In [ ]:
predict(model, test_dataloader)

100%|██████████| 70/70 [00:10<00:00,  6.90it/s]


{0}